In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%pip install -q spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
%pip install -q pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Config:
  spurious_correlation_strength  = 0.8
  spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
  label_noise = 0.001
  size = 28
  flip_probability = 0.5
  batch_size = 256
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 1e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

## Dataset

In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="train",
    label_noise=Config.label_noise
)
trainset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",

    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="test",
    label_noise=Config.label_noise
)
valset.initialize()


100%|██████████| 9912422/9912422 [00:00<00:00, 102694872.99it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 113105222.99it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31614347.35it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19990061.67it/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw




100%|██████████| 10000/10000 [00:01<00:00, 7319.80it/s]


In [7]:

## Augmentation
from torchvision.transforms import v2

size = 28
train_transform = transforms.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    v2.ToImage(),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [8]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = self.transform(img)
        return (img, augmented_img), label

In [9]:
train_dataset = SpucoDataset(trainset, train_transform)
val_dataset = SpucoDataset(valset, test_transform)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=False)}

In [10]:
from IPython.display import display
for (k, v) in valset.group_partition.items():
  print(f"Group: {k}, Qty: {len(v)}")
  img = T.ToPILImage()(valset[v[0]][0])
  display(img)

Group: (3, 4), Qty: 397


Group: (1, 2), Qty: 408


Group: (0, 2), Qty: 423


Group: (0, 3), Qty: 423


Group: (2, 3), Qty: 375


...

Group: (2, 0), Qty: 375


Group: (3, 3), Qty: 397


Group: (4, 2), Qty: 397


Group: (1, 4), Qty: 408


Group: (4, 1), Qty: 397


In [11]:
T.ToPILImage()(valset[v[0]][0])

## Contrastive Loss

In [12]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)
criterion = losses.SelfSupervisedLoss(criterion)

## Model

In [13]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [14]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [15]:
import os
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    val_loss = 0
    for batch_idx, (images, _) in enumerate(dataloaders['train']):
        batch_tot_loss = 0
        if (batch_num % 50 == 1):
          print("in batch", batch_num)
        batch_num=batch_num+1
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)
        optimizer.zero_grad()

        images_emb = clr_model(images)
        augmented_images_emb = clr_model(augmented_images)
        loss = criterion(images_emb, augmented_images_emb)

        loss.backward()

        optimizer.step()
        tot_loss += loss.item()
        batch_tot_loss += loss.item()
        if (batch_idx % 200 == 1):
            print('batch loss=', batch_tot_loss)
    scheduler.step()



    # Validation
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(dataloaders['eval']):
            val_batch_loss = 0
            if (batch_num % 50 == 1):
                print("in batch", batch_num)
            batch_num=batch_num+1
            images, augmented_images = images[0], images[1]
            images, augmented_images = images.to(device), augmented_images.to(device)


            images_emb = clr_model(images)
            augmented_images_emb = clr_model(augmented_images)
            loss = criterion(images_emb, augmented_images_emb)



            val_loss += loss.item()
            val_batch_loss += loss.item()
        if (batch_idx % 200 == 1):
          print('batch loss=', val_batch_loss)

    print(f"epoch {epoch}train loss= {tot_loss/len(dataloaders['train'])}")
    print(f"epoch {epoch}val loss= {val_loss/len(dataloaders['train'])}")

    if epoch % 10 ==0:
        save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
        if not os.path.isdir(save_dir):
          os.mkdir(save_dir)
        modelstate_save_name = f'modelstate_bs_restart_r_{Config.spurious_correlation_strength}_LARGE_epoch{epoch}.pt'
        path_modelstate = os.path.join(save_dir, modelstate_save_name)
        torch.save(clr_model.state_dict(), path_modelstate)

        parameters_end = {
            'net': clr_model.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
        }
        paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
        path_paramsend = os.path.join(save_dir, paramsend_save_name)
        torch.save(parameters_end, path_paramsend)



In [16]:
start_epoch=1
max_epoch=30

for epoch in range(start_epoch, start_epoch+max_epoch):
  train(epoch)


Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 6.2646989822387695
in batch 51
in batch 101
in batch 151
in batch 201
epoch 1train loss= 5.720206882091278
epoch 1val loss= 1.1902921199798584

Epoch: 2


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 5.320600509643555
in batch 51
in batch 101
in batch 151
in batch 201
epoch 2train loss= 5.245260712948252
epoch 2val loss= 1.1267439405968849

Epoch: 3


...

in batch 1
batch loss= 4.93448543548584
in batch 51
in batch 101
in batch 151
in batch 201
epoch 28train loss= 4.9055869579315186
epoch 28val loss= 1.0307187907239224

Epoch: 29


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.932932376861572
in batch 51
in batch 101
in batch 151
in batch 201
epoch 29train loss= 4.905400113856539
epoch 29val loss= 1.0280094121364838

Epoch: 30


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.900737762451172
in batch 51
in batch 101
in batch 151
in batch 201
epoch 30train loss= 4.894201605877978
epoch 30val loss= 1.0330384969711304


In [17]:
## Load the pre-trained model parameters
paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
path_paramsend = path_paramsend = os.path.join(save_dir, paramsend_save_name)
checkpoint = torch.load(path_paramsend, map_location = torch.device(device))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=epoch+1
clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

## PCA and 2D Scatter Plot

In [18]:
torch.cuda.empty_cache()

In [19]:
## https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA


torch.cuda.empty_cache()
device = 'cpu'
clr_model.to(device)

embeddings = []
targets = []

for batch_idx, ((images, _), labels) in enumerate(dataloaders['eval']):
  embeddings.append(clr_model(images).detach().cpu())
  targets.append(labels)
  torch.cuda.empty_cache()


# clr_model = None

embeddings = torch.cat(embeddings).detach().cpu().numpy()
targets = torch.cat(targets).cpu().numpy()

In [20]:
import pandas as pd
comp = 100
pca_2d = PCA(n_components=comp)
PCs_2d = pd.DataFrame(pca_2d.fit_transform(embeddings))
PCs_2d.columns = [f"PC{i+ 1}" for i in range(comp)]
pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')

In [21]:
pce = pca_2d.fit_transform(embeddings)

In [22]:
pca_2d.singular_values_

array([516.502    , 510.19864  , 483.20673  , 447.18988  , 358.596    ,
       269.77896  , 219.65678  , 181.69867  , 161.45027  ,  56.456142 ,
        18.199669 ,  14.734824 ,  11.294672 ,  10.585806 ,   9.825913 ,
         9.465994 ,   9.012622 ,   8.797796 ,   8.135482 ,   7.9049916,
         7.6730266,   7.4432917,   7.233722 ,   6.9077783,   6.6840754,
         6.4304137,   6.2673264,   6.0698085,   5.8796163,   5.767646 ,
         5.5378222,   5.40106  ,   5.178099 ,   5.0688963,   4.9142337,
         4.7297654,   4.708108 ,   4.558324 ,   4.4495625,   4.35429  ,
         4.3009176,   4.1577287,   3.9641252,   3.9342241,   3.9031806,
         3.800961 ,   3.675992 ,   3.631605 ,   3.5922198,   3.479842 ,
         3.4194562,   3.3371038,   3.258861 ,   3.2330997,   3.2117167,
         3.1710725,   3.0722082,   3.0319817,   2.960778 ,   2.8979354,
         2.840851 ,   2.8097024,   2.757215 ,   2.6846046,   2.650763 ,
         2.6103144,   2.52716  ,   2.4874084,   2.4289222,   2.4

In [23]:
pca_embs['true_label'] = targets

In [24]:
clusters = {k : np.stack([PCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [25]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

traces = []

for (k, v) in sorted(clusters.items()):
  traces.append(go.Scatter(
                    x = v[:, 2],
                    y = v[:, 3],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))
data = traces

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [26]:
for (true, spur), idxs in valset.group_partition.items():
  for i in idxs:
    pca_embs.loc[i, 'spurious_label'] = spur
pca_embs['spurious_label'] = pca_embs['spurious_label'].astype(int)

In [27]:
valset.group_partition.keys()

dict_keys([(3, 4), (1, 2), (0, 2), (0, 3), (2, 3), (2, 1), (4, 0), (2, 4), (4, 3), (3, 0), (4, 4), (0, 4), (2, 2), (3, 2), (1, 0), (0, 1), (3, 1), (0, 0), (1, 3), (1, 1), (2, 0), (3, 3), (4, 2), (1, 4), (4, 1)])

In [28]:
true_label_clusters = {
    # True label: {group, cluster_dim}
}
for l in pca_embs['true_label'].unique():
  true_cluster = pca_embs[pca_embs['true_label'] == l]
  true_emb = true_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,true_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  true_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [29]:
spurious_label_clusters = {
    # Spurious label: {group, cluster_dim}
}
for l in pca_embs['spurious_label'].unique():
  spurious_cluster = pca_embs[pca_embs['spurious_label'] == l]
  spurious_emb = spurious_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,spurious_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  spurious_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [30]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in true_label_clusters.items():
  traces = []
  for s in np.sort(v['spurious_label'].unique()):
    group = v[v['spurious_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC2_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"Subgroup {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - True Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

In [31]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in spurious_label_clusters.items():
  traces = []
  for s in np.sort(v['true_label'].unique()):
    group = v[v['true_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC1_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"True Label {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - Spurious Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_Spurious_Label_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

## t-SNE

In [32]:
from sklearn.manifold import TSNE

perplexity = 50

#T-SNE with two dimensions
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

TCs_2d = pd.DataFrame(tsne_2d.fit_transform(embeddings))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

In [33]:
tsne_embs

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,TC1_2d,TC2_2d
0,-0.711962,-0.056097,-0.695458,0.798552,1.682121,0.331118,0.338566,-1.234134,0.367142,0.646287,...,-0.727041,-0.616065,-1.354068,-0.447976,-0.766591,1.408877,-1.104561,-0.109808,1.782105,-37.055176
1,1.087621,-0.111185,-0.535210,-0.775946,-0.651492,1.012852,-0.812605,0.664143,0.455196,0.505159,...,1.324095,1.345927,0.433395,1.178420,1.552723,0.917637,0.421815,0.577735,44.252048,-11.999149
2,1.388605,-0.394574,-0.630379,-0.472738,-0.366459,1.291185,-0.534777,0.438735,0.150552,0.536793,...,1.566573,1.381812,-0.146617,0.763550,1.699377,0.889292,-0.397682,1.376057,48.267822,-37.319832
3,-1.179209,1.726182,-0.229607,-0.357431,-0.852759,-0.031431,-1.445424,0.757690,0.204283,0.582811,...,-0.206898,-0.323236,-0.140122,-0.357691,-1.455380,-1.830578,0.950629,-0.862228,30.069210,51.152149
4,-1.042594,0.051861,-1.019507,-0.940901,-0.987430,0.727014,-1.259526,0.974408,0.321685,0.789089,...,0.219508,-1.135953,-0.338162,-0.071314,-0.950796,-0.738913,0.255397,-0.154046,28.567225,25.298973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.514475,0.689298,0.368241,1.021516,1.469901,0.269890,0.172533,-1.726358,0.361885,0.917706,...,-0.722599,0.117735,-1.392472,-0.254273,-0.238606,1.188760,-0.869506,0.157476,-9.606159,-14.428977
9996,-0.377816,0.147105,1.447801,-1.455624,-1.730621,-2.148420,0.249610,1.097020,1.315064,-1.029810,...,-1.726395,-0.532016,1.683402,1.599023,0.218076,-0.574136,2.033728,0.557283,-13.225377,67.877884
9997,-1.850828,-0.887296,-1.790782,-0.885494,-0.609522,1.072792,-0.891883,1.099493,0.016003,1.037148,...,-0.034638,-2.986960,-1.479014,-1.027623,-1.609683,-0.671077,-1.383036,1.311814,39.055820,10.527350
9998,1.034567,-0.773104,-1.070115,-0.863084,-0.445319,1.711599,-1.015941,0.592459,0.314023,0.959145,...,1.838547,0.831965,-0.041542,1.262465,1.850360,1.859753,-0.380506,1.334300,59.606041,-26.447172


In [34]:
tsne_clusters = {k : np.stack([TCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [35]:
tsne_traces = []

for (k, v) in sorted(tsne_clusters.items()):
  tsne_traces.append(go.Scatter(
                    x = v[:, 0],
                    y = v[:, 1],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

In [36]:
data = tsne_traces

title = "Visualizing Clusters in Two Dimensions Using t-SNE"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)